# 전처리 1

In [1]:
import pandas as pd
import numpy as np
import re
import copy
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
pd.options.mode.chained_assignment = None  # 경고 숨기기

In [2]:
# 데이터 불러오기
news = pd.read_csv('hybe_news.csv')
news

,Title,Date
0,"하이브, 새 걸그룹 선보인다…데뷔 서바이벌 프로그램 론칭",2023.06.01
1,BTS '단체활동 중단' 미리 알고…하이브 직원들 주식매도 적발 [1분뉴스],2023.06.01
2,"미공개정보 이용한 하이브 직원 후회막심?...SM사태 이후 하이브 급등, 카카오 하락",2023.06.01
3,다 오를 때 혼자 부진했는데…'엑소'와의 갈등에 또 다시 악재 맞은 SM,2023.06.01
4,“하이브만이 아니었네”…올해 주식 불공정거래 절반이 '미공개정보 이용',2023.06.01
...,...,...
9274,"[MD포토] 이희준, '완벽한 핸섬가이'",2024.05.27
9275,뉴진스 앨범 첫날 81만장…민희진 해임 가처분 이번주 결론,2024.05.27
9276,"민희진 공격 vs 아일릿 보호…‘뉴진스 표절’ 둔 고소전, 어떻게 봐야할까 [D:이슈]",2024.05.27
9277,"[MD포토] 공승연, '정말 재밌어요~'",2024.05.27


### 1. Date 컬럼 형식 맞추기 - '실제 주식 데이터'와 비교 위해서

In [3]:
### 1. Date 컬럼 형식 맞추기 - '실제 주식 데이터'와 비교 위해서
news['Date'] = pd.to_datetime(news['Date'], format='%Y.%m.%d').dt.strftime('%Y-%m-%d')
# 컬럼 순서 변경
news = news[['Date', 'Title']]
news

,Date,Title
0,2023-06-01,"하이브, 새 걸그룹 선보인다…데뷔 서바이벌 프로그램 론칭"
1,2023-06-01,BTS '단체활동 중단' 미리 알고…하이브 직원들 주식매도 적발 [1분뉴스]
2,2023-06-01,"미공개정보 이용한 하이브 직원 후회막심?...SM사태 이후 하이브 급등, 카카오 하락"
3,2023-06-01,다 오를 때 혼자 부진했는데…'엑소'와의 갈등에 또 다시 악재 맞은 SM
4,2023-06-01,“하이브만이 아니었네”…올해 주식 불공정거래 절반이 '미공개정보 이용'
...,...,...
9274,2024-05-27,"[MD포토] 이희준, '완벽한 핸섬가이'"
9275,2024-05-27,뉴진스 앨범 첫날 81만장…민희진 해임 가처분 이번주 결론
9276,2024-05-27,"민희진 공격 vs 아일릿 보호…‘뉴진스 표절’ 둔 고소전, 어떻게 봐야할까 [D:이슈]"
9277,2024-05-27,"[MD포토] 공승연, '정말 재밌어요~'"


# 2. 텍스트 정제된 ppc_Title 컬럼 만들기

In [4]:
### 2. 텍스트 정제
def preprocess_title(text):  
    text = text.upper()  # (1) 대문자로 변환

    # (2) 대괄호 및 그 안의 내용 제거,  [1분뉴스], [MD포토]등
    text = re.sub(r'\[.*?\]', ' ', text)   
    
    # (3) K-팝, K-POP, KPOP을 다 같은 K팝으로 대체
    text = text.replace('K-팝', "K팝") 
    text = text.replace('K-POP', "K팝")
    text = text.replace('KPOP', "K팝")
    text = text.replace('케이팝', "K팝")

    # (4) 특수 문자가 들어간 고유명사 "(여자)아이들" 을 임시로 다른 문자열("ABCD")로 대체
    text = text.replace("(여자)아이들", "ABCD")

    # (5) 특수 문자 제거 및 공백 유지
    text = re.sub(r'[^\w\s]', ' ', text)

    # (6) 한글과 영어 빼고 다른 문자 삭제 (한자, 숫자)
    text = re.sub(r'[^가-힣A-Za-z\s]', '', text)

    # (7) 대체된 문자열("ABCD")을 원래 문자열 "(여자)아이들"로 복원
    text = text.replace("ABCD", '(여자)아이들')
        
    # (8) 문자열 앞뒤 공백 제거
    text = text.strip()
    return text

# 'Title' 컬럼에 텍스트 정제 함수 적용
news['ppc_Title'] = news['Title'].apply(preprocess_title)

In [5]:
news

,Date,Title,ppc_Title
0,2023-06-01,"하이브, 새 걸그룹 선보인다…데뷔 서바이벌 프로그램 론칭",하이브 새 걸그룹 선보인다 데뷔 서바이벌 프로그램 론칭
1,2023-06-01,BTS '단체활동 중단' 미리 알고…하이브 직원들 주식매도 적발 [1분뉴스],BTS 단체활동 중단 미리 알고 하이브 직원들 주식매도 적발
2,2023-06-01,"미공개정보 이용한 하이브 직원 후회막심?...SM사태 이후 하이브 급등, 카카오 하락",미공개정보 이용한 하이브 직원 후회막심 SM사태 이후 하이브 급등 카카오 하락
3,2023-06-01,다 오를 때 혼자 부진했는데…'엑소'와의 갈등에 또 다시 악재 맞은 SM,다 오를 때 혼자 부진했는데 엑소 와의 갈등에 또 다시 악재 맞은 SM
4,2023-06-01,“하이브만이 아니었네”…올해 주식 불공정거래 절반이 '미공개정보 이용',하이브만이 아니었네 올해 주식 불공정거래 절반이 미공개정보 이용
...,...,...,...
9274,2024-05-27,"[MD포토] 이희준, '완벽한 핸섬가이'",이희준 완벽한 핸섬가이
9275,2024-05-27,뉴진스 앨범 첫날 81만장…민희진 해임 가처분 이번주 결론,뉴진스 앨범 첫날 만장 민희진 해임 가처분 이번주 결론
9276,2024-05-27,"민희진 공격 vs 아일릿 보호…‘뉴진스 표절’ 둔 고소전, 어떻게 봐야할까 [D:이슈]",민희진 공격 VS 아일릿 보호 뉴진스 표절 둔 고소전 어떻게 봐야할까
9277,2024-05-27,"[MD포토] 공승연, '정말 재밌어요~'",공승연 정말 재밌어요


# 3. ppc_Title 컬럼 중복 확인 및 제거

In [6]:
### 3. 전처리된 기사 제목 ppc_Title 기준 중복 행 삭제, 첫번째 발견된 행만 유지
news = news.drop_duplicates(subset='ppc_Title', keep='first')
news.shape # 9279 -> 9192

(9192, 3)

# 4. 띄어쓰기 기준 명사만 추출하여 Tokens 컬럼 만들기

In [7]:
### 4. 띄어쓰기 후 각 단어 형태소 분석 -> 명사만 추출
kiwi = Kiwi(typos='basic')
stopwords = Stopwords()

def tokenize_text(text):
    words = text.split()  # (1) 띄어쓰기로 텍스트 나누기
    token_list = []
    for word in words:
        tokens = kiwi.tokenize(word)  # (2) 각 단어를 형태소 분석
        for token in tokens:
            if token.tag in ['NNG', 'NNP']:   # (3) 명사(NNG, NNP)만 추출
                token_list.append(token.form)
    return token_list
news['Tokens'] = news['ppc_Title'].apply(tokenize_text)

# <span style="color:pink"> --- 중간 저장 -------------

In [8]:
# CSV 파일로 저장
news.to_csv('hybe_01.Preprocessing_01.csv', index=False)

In [9]:
news.head()

,Date,Title,ppc_Title,Tokens
0,2023-06-01,"하이브, 새 걸그룹 선보인다…데뷔 서바이벌 프로그램 론칭",하이브 새 걸그룹 선보인다 데뷔 서바이벌 프로그램 론칭,"[하이브, 걸그룹, 데뷔, 서바이벌, 프로그램, 론칭]"
1,2023-06-01,BTS '단체활동 중단' 미리 알고…하이브 직원들 주식매도 적발 [1분뉴스],BTS 단체활동 중단 미리 알고 하이브 직원들 주식매도 적발,"[단체, 활동, 중단, 하이브, 직원, 주식, 매도, 적발]"
2,2023-06-01,"미공개정보 이용한 하이브 직원 후회막심?...SM사태 이후 하이브 급등, 카카오 하락",미공개정보 이용한 하이브 직원 후회막심 SM사태 이후 하이브 급등 카카오 하락,"[공개, 정보, 이용, 하이브, 직원, 후회, 막심, 사태, 이후, 하이브, 급등,..."
3,2023-06-01,다 오를 때 혼자 부진했는데…'엑소'와의 갈등에 또 다시 악재 맞은 SM,다 오를 때 혼자 부진했는데 엑소 와의 갈등에 또 다시 악재 맞은 SM,"[때, 부진, 엑소, 와, 갈등, 악재]"
4,2023-06-01,“하이브만이 아니었네”…올해 주식 불공정거래 절반이 '미공개정보 이용',하이브만이 아니었네 올해 주식 불공정거래 절반이 미공개정보 이용,"[하이브, 올해, 주식, 공정, 거래, 절반, 공개, 정보, 이용]"


# 5. 각 행의 리스트를 문자열로 결합하고 컴마 제거


In [10]:
import pandas as pd 
news_copy = pd.read_csv('hybe_01.Preprocessing_01.csv')

In [11]:
# Tokens 열을 문자열로 변환
news_copy['Tokens'] = news_copy['Tokens'].astype(str)

In [12]:
# ### 5. Tokens 열에서 리스트 기호와 쉼표 제거
# news['Tokens'] = news['Tokens'].str.replace("[\[\]']", "", regex=True)
# news['Tokens'] = news['Tokens'].str.replace(",", "", regex=False)
# news

In [13]:
### 5. Tokens 열에서 리스트 기호와 쉼표 제거
news_copy['Tokens'] = news_copy['Tokens'].str.replace("[\[\]']", "", regex=True)
news_copy['Tokens'] = news_copy['Tokens'].str.replace(",", "", regex=False)
news_copy

,Date,Title,ppc_Title,Tokens
0,2023-06-01,"하이브, 새 걸그룹 선보인다…데뷔 서바이벌 프로그램 론칭",하이브 새 걸그룹 선보인다 데뷔 서바이벌 프로그램 론칭,하이브 걸그룹 데뷔 서바이벌 프로그램 론칭
1,2023-06-01,BTS '단체활동 중단' 미리 알고…하이브 직원들 주식매도 적발 [1분뉴스],BTS 단체활동 중단 미리 알고 하이브 직원들 주식매도 적발,단체 활동 중단 하이브 직원 주식 매도 적발
2,2023-06-01,"미공개정보 이용한 하이브 직원 후회막심?...SM사태 이후 하이브 급등, 카카오 하락",미공개정보 이용한 하이브 직원 후회막심 SM사태 이후 하이브 급등 카카오 하락,공개 정보 이용 하이브 직원 후회 막심 사태 이후 하이브 급등 카카오 하락
3,2023-06-01,다 오를 때 혼자 부진했는데…'엑소'와의 갈등에 또 다시 악재 맞은 SM,다 오를 때 혼자 부진했는데 엑소 와의 갈등에 또 다시 악재 맞은 SM,때 부진 엑소 와 갈등 악재
4,2023-06-01,“하이브만이 아니었네”…올해 주식 불공정거래 절반이 '미공개정보 이용',하이브만이 아니었네 올해 주식 불공정거래 절반이 미공개정보 이용,하이브 올해 주식 공정 거래 절반 공개 정보 이용
...,...,...,...,...
9187,2024-05-27,"[MD포토] 이희준, '완벽한 핸섬가이'",이희준 완벽한 핸섬가이,이희주 완벽 핸섬가
9188,2024-05-27,뉴진스 앨범 첫날 81만장…민희진 해임 가처분 이번주 결론,뉴진스 앨범 첫날 만장 민희진 해임 가처분 이번주 결론,뉴진스 앨범 첫날 만장 민희 진 해임 처분 이번 주 결론
9189,2024-05-27,"민희진 공격 vs 아일릿 보호…‘뉴진스 표절’ 둔 고소전, 어떻게 봐야할까 [D:이슈]",민희진 공격 VS 아일릿 보호 뉴진스 표절 둔 고소전 어떻게 봐야할까,민희 진 공격 아일릿 보호 뉴진스 표절 고소 전
9190,2024-05-27,"[MD포토] 공승연, '정말 재밌어요~'",공승연 정말 재밌어요,공승


In [14]:
# CSV 파일로 저장
news_copy.to_csv('hybe_01.Preprocessing_02.csv', index=False)

---

# 6. 결측치 확인 및 제거

In [15]:
import pandas as pd 
news_check = pd.read_csv('hybe_01.Preprocessing_02.csv')
news_check.head()


,Date,Title,ppc_Title,Tokens
0,2023-06-01,"하이브, 새 걸그룹 선보인다…데뷔 서바이벌 프로그램 론칭",하이브 새 걸그룹 선보인다 데뷔 서바이벌 프로그램 론칭,하이브 걸그룹 데뷔 서바이벌 프로그램 론칭
1,2023-06-01,BTS '단체활동 중단' 미리 알고…하이브 직원들 주식매도 적발 [1분뉴스],BTS 단체활동 중단 미리 알고 하이브 직원들 주식매도 적발,단체 활동 중단 하이브 직원 주식 매도 적발
2,2023-06-01,"미공개정보 이용한 하이브 직원 후회막심?...SM사태 이후 하이브 급등, 카카오 하락",미공개정보 이용한 하이브 직원 후회막심 SM사태 이후 하이브 급등 카카오 하락,공개 정보 이용 하이브 직원 후회 막심 사태 이후 하이브 급등 카카오 하락
3,2023-06-01,다 오를 때 혼자 부진했는데…'엑소'와의 갈등에 또 다시 악재 맞은 SM,다 오를 때 혼자 부진했는데 엑소 와의 갈등에 또 다시 악재 맞은 SM,때 부진 엑소 와 갈등 악재
4,2023-06-01,“하이브만이 아니었네”…올해 주식 불공정거래 절반이 '미공개정보 이용',하이브만이 아니었네 올해 주식 불공정거래 절반이 미공개정보 이용,하이브 올해 주식 공정 거래 절반 공개 정보 이용


In [16]:
news_check[news_check['Tokens'].isna()]

,Date,Title,ppc_Title,Tokens
1829,2023-08-09,[코스피·코스닥 전 거래일(8일) 주요공시],NaN,NaN
6946,2024-02-28,[오늘의 보안 영어] in tandem with,IN TANDEM WITH,NaN


In [17]:
### 6. 결측치 제거
news = news.dropna(subset=['Tokens'])
news[news['Tokens'].isna()]

,Date,Title,ppc_Title,Tokens


In [18]:
### 6. 결측치 제거
news_check = news_check.dropna(subset=['Tokens'])
news_check[news_check['Tokens'].isna()]

,Date,Title,ppc_Title,Tokens


In [21]:
# CSV 파일로 저장
news_check.to_csv('hybe_01.Preprocessing_FIN.csv', index=False)

---

In [22]:
# 잘 저장 됐는지 확인
import pandas as pd 
news_check2 = pd.read_csv('hybe_01.Preprocessing_FIN.csv')

In [23]:
news_check2[news_check2['Tokens'].isna()]

,Date,Title,ppc_Title,Tokens
